In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
## Load Dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [15]:
## Preprocess the data
# Drop unnecessary columns
dta=data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
dta

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [16]:
## Encoding categorical variables
label_encoder_gender = LabelEncoder()
dta['Gender'] = label_encoder_gender.fit_transform(dta['Gender'])   
dta


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [17]:
## one-hot encoding for 'Geography' column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
geography_encoded = onehot_encoder.fit_transform(dta[['Geography']])
geography_encoded

array([[0., 0.],
       [0., 1.],
       [0., 0.],
       ...,
       [0., 0.],
       [1., 0.],
       [0., 0.]])

In [18]:
## Combine onehot encoded column with original dataframe
pd.DataFrame(geography_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))    
dta_final = pd.concat([dta.drop('Geography', axis=1), pd.DataFrame(geography_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))], axis=1)
dta_final

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,1.0,0.0


In [38]:
## save thencoder and scaler
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)
with open('onehot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(onehot_encoder, f)  
scaler = StandardScaler()

In [40]:
## Divide the data into features and target variable
X = dta_final.drop('Exited', axis=1)
y = dta_final['Exited']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale these features
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.36766974,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ...,  1.6612541 ,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.25280688,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ..., -0.1427649 ,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ..., -0.05082558,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.81456811,
         1.72572313, -0.57638802]])

In [41]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [42]:
dta_final

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,1.0,0.0


In [43]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping


In [44]:
X_train.shape[1]

11

In [45]:
# build the ANN model
model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 connected to input layer
    Dense(32, activation='relu'),  ## HL2 connected to HL1
    Dense(1, activation='sigmoid')  ## output layer    
])
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
opt=tf.keras.optimizers.Adam(learning_rate=0.001)
# compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])


In [46]:
##set up the TensorBoard 
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [47]:
## set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [48]:
## Train the model
history = model.fit(
    X_train, y_train, 
    validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback, early_stopping])

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.4495 - accuracy: 0.8086 - val_loss: 0.4002 - val_accuracy: 0.8355
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3902 - accuracy: 0.8380 - val_loss: 0.3651 - val_accuracy: 0.8450
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3608 - accuracy: 0.8500 - val_loss: 0.3518 - val_accuracy: 0.8540
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3470 - accuracy: 0.8564 - val_loss: 0.3507 - val_accuracy: 0.8565
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3415 - accuracy: 0.8583 - val_loss: 0.3428 - val_accuracy: 0.8580
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3373 - accuracy: 0.8621 - val_loss: 0.3573 - val_accuracy: 0.8560
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3360 - accuracy: 0.8596 - val_loss: 0.3422 - val_accuracy: 0.8605

In [49]:
model.save('churn_model.h5')

c:\Users\anagh\RNN_Project_Churn_Modeling\tfenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [50]:
## Load tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [51]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 12464), started 0:20:39 ago. (Use '!kill 12464' to kill it.)